# 7.2 Contour Feature
- There are several different features of contours, like area, perimeter, centroid, bounding box etc.
    - Aspect Ratio
    - Extent 
    - Circularity
    - Orientation
    <br><br>

In [3]:
import cv2
import numpy as np

<br><br><br>
- **Aspect Ratio**
    - It is the ratio of width to height of bounding rect of the object.<br>
    ```python
    x, y, w, h = cv2.boundingRect(cnt)
    aspect_ratio = w/h
    ``` 
<br><br>
- **Extent**
    - Extent is the ratio of contour area to bounding rectangle area.<br>
    ```python
    area = cv2.contourArea(cnt)
    x, y, w, h = cv2.boundingRect(cnt)
    rect_area = w*h
    extent = area/rect_area
    ```
<br><br>
- **Circularity**
    - Measure of how close a contour is to a perfect circle. Ranges from 0 to 1 (1 = perfect circle).<br>
    ```python
    area = cv2.contourArea(cnt)
    perimeter = cv2.arcLength(cnt, True)
    circularity = 4 * np.pi * area / (perimeter * perimeter)
    ```
    - Use to filter round shapes (e.g., coins, dots); values < ~0.7 often indicate elongated/irregular shapes.
<br><br><br>
<br><br>
- **Orientation**
    - Orientation (angle) of the contour's major axis. Can be obtained from rotated bounding rectangle.<br>
    ```python
    (cx, cy), (w, h), angle = cv2.minAreaRect(cnt)

    # OpenCV returns angle in [0°, 90°); it is the box rotation.
    # Angle is defined with respect to the rectangle’s width side, 
    # Angle direction is clockwise from positive X-axis.
    # To get the long-axis orientation:
    if h >= w  :
        angle = angle + 90

    # Convert to counter-clockwise from positive X-axis
    # % 360 is used to keep angle within [0, 360)
    angle_ccw = (180 - angle) % 360
    ```


<br><br><br>
- Example Updated <font color="orange">improved shape detection code</font> from notebook `7.1 contour_feature.ipynb` by formalizing formula based on above contour properties definitions.

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    # calculate contour area
    area = cv2.contourArea(contour)

    # approximate contour to polygon with epsilon = 1% (0.01) of arc length
    perimeter = cv2.arcLength(contour, True)
    epsilon = 0.01 * perimeter
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Number of vertices of approximated polygon
    vertices = len(approx)


    # Shape classification based on number of vertices
    if vertices == 3: # 3 sides
        shape = "Triangle" 


    elif vertices == 4: # 4 sides
        x, y, w, h = cv2.boundingRect(approx)
        rect_area = w * h
        extent = area / rect_area
        aspect_ratio = w / h

        # if extent is close to 1, then it's square or rectangle
        if extent > 0.9:

            # if aspect_ratio is almost equal to 1, then it's square
            if aspect_ratio >= 0.95 and aspect_ratio <= 1.05: 
                shape = "Square"
            else:
                shape = "Rectangle"

        else :
            shape = "Quadrilateral" # trapezoid, kite, skewed quad, etc.


    elif vertices > 4: # more than 4 sides
        # compute circularity, if value is between 0 and 1, where 1 means perfect circle
        circularity = 4 * np.pi * area / (perimeter * perimeter)

        if circularity > 0.85: # close to 1, then it's circle
            shape = "Circle" 
        else:
            shape = "Polygon"

    else:
        shape = "Unknown"

    # Draw approximated contour (polygon)
    cv2.drawContours(img, [approx], -1, (255, 0, 255), 2)

    # find bounding rectangle, to position the shape name text
    x, y, w, h = cv2.boundingRect(approx)

    # Draw shape name
    cv2.putText(
        img,
        shape,
        (x, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.4,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
- Example detect object orientation angle using `cv2.minAreaRect()` function on `blocks_rotated.jpg` image.

In [24]:
# read the image "blocks_rotated.jpg"
img = cv2.imread("blocks_rotated.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    (cx, cy), (min_x, min_y), angle = cv2.minAreaRect(contour)

    # Draw angle
    cv2.putText(
        img,
        f"angle: {angle:.0f}",
        (int(cx) - 20, int(cy)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )
     # Draw size
    cv2.putText(
        img,
        f"w:{min_x:.0f} h:{min_y:.0f}",
        (int(cx) - 20, int(cy) + 24),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- ⚠️⚠️⚠️ How width is selected (important)

    - width = length of the rectangle side whose direction is used to define angle
    - So:
        - The side used to compute the **angle** becomes **width**
        - The **perpendicular** side becomes **height**
    - There is NO guarantee that:
        - width ≥ height
        - width is horizontal
        - width is the longer side
<img src="res/min_rect_area_angle.png" alt="minAreaRect_angle_definition" width="700"/>

- To get the <font color="orange">long-axis orientation</font>, we check if <font color="orange">height >= width</font>, then we <font color="orange">add 90°</font> to the angle.
- below is example code to get the long-axis orientation angle.
- remember the angle direction is <font color="orange">clockwise</font> from positive X-axis.

In [34]:
# read the image "blocks_rotated.jpg"
img = cv2.imread("blocks_rotated.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    (cx, cy), (min_w, min_h), angle = cv2.minAreaRect(contour)

    # To get the long-axis orientation:
    if min_h >= min_w  :
        angle = angle + 90

    # Draw angle
    cv2.putText(
        img,
        f"angle: {angle:.0f}",
        (int(cx) - 20, int(cy)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )
     # Draw size
    cv2.putText(
        img,
        f"w:{min_x:.0f} h:{min_y:.0f}",
        (int(cx) - 20, int(cy) + 24),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Now the angle represents the orientation of the long side of the rectangle.<br>
<img src="res/min_rect_area_angle_long_side.png"  width="700"/>

<br><br><br>
- To make angle direction <font color="orange">counter-clockwise from positive X-axis of long side</font>, we can do the following modification,
```python
    if min_h >= min_w  :
        angle = angle + 90

    # Convert to counter-clockwise from positive X-axis
    # % 360 is used to keep angle within [0, 360)
    angle_ccw = (180 - angle) % 360
```

- Example computing orientation angle of long side in counter-clockwise direction from positive X-axis.

In [38]:
# read the image "blocks_rotated.jpg"
img = cv2.imread("blocks_rotated.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    (cx, cy), (min_w, min_h), angle = cv2.minAreaRect(contour)

    # To get the long-axis orientation:
    if min_h >= min_w  :
        angle = angle + 90

    # Convert to counter-clockwise from positive X-axis
    # % 360 is used to keep angle within [0, 360)
    angle_ccw = (180 - angle) % 360

    # Draw angle
    cv2.putText(
        img,
        f"angle: {angle_ccw:.0f}",
        (int(cx) - 20, int(cy)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )
     # Draw size
    cv2.putText(
        img,
        f"w:{min_x:.0f} h:{min_y:.0f}",
        (int(cx) - 20, int(cy) + 24),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Now the angle represents the orientation of the long side of the rectangle in counter-clockwise direction from positive X-axis.<br>
<img src="res/min_rect_area_angle_long_side_ccw.png"  width="700"/>